<a href="https://colab.research.google.com/github/Dennis-Farias/aulaInteligenciaArtificial/blob/main/Buscas_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Busca com subida da encosta(Hill Clibing)</h1>
<h2>Problema das N Rainhas</h2>
<p><b>Em um tabuleiro NxN deve ter N rainhas que não se atacam</b></p>
<p> F = Número de rainhas que não se atacam <br>
    Mínimo = 0 (Meta)<br>
    Máximo = Combinação de N 2 a 2 - COMB(N,2)<br>
    Para N = 8 <br>
    MAX = 8! / ((8-2)!*2!)-8*7/2 = 28 pares de rainhas que não se atacam
</p>

In [16]:
# Configuração inicial
import logging as log
import sys


log.basicConfig(format='%(levalname)s : %(message)s', stream=sys.stdout)
root = log.getLogger()
root.setLevel(log.DEBUG)

logi = log.info
loge= log.error
logw = log.warning
logd = log.debug

In [17]:
# import das bibliotecas
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (14,7)
import math
import random
import copy
import itertools as it
TAMANHOTABULEIRODEFAULT = 8
DIRECAO =['norte','nordeste','leste','sudeste','sul', 'sudoeste','oeste','noroeste'] # Movimentos que podem ser feitos pela rainha

In [18]:
# Gerar tabuleiro com matriz t por t de zeros com 1 (rainhas) na primeira coluna
def geraTabuleiro(tamanho):
    if tamanho<2: return[-1]
    tab = []
    for i in range(tamanho):
        l=[]
        for j in range(tamanho):
            l.append(0)
        tab.append(l)
    for i in range(tamanho): tab[i][0]=1
    return tab

In [19]:
# Gera tabuleiro posicionando as rainhas aleatoriamente
def geraTabuleiroAleatorio(tamanho):
    if tamanho<2: return[-1]
    tab = []
    for i in range(tamanho):
        l=[]
        for j in range(tamanho):
            l.append(0)
        tab.append(l)
    for i in range(tamanho):
      c = random.randint(0,tamanho-1)
      tab[i][c]=1
    return tab

In [20]:
def printTabuleiro(tab):
    for i in tab:print(i)

In [ ]:
# Testar impressão do tabuleiro
printTabuleiro(geraTabuleiroAleatorio(8))

In [ ]:
# Testar impressão do tabuleiro
printTabuleiro(geraTabuleiro(8))

In [ ]:
# Testar impressão do tabuleiro
printTabuleiro(geraTabuleiro(16))

In [24]:
# Localiza rainhas
def locateQueens(estado):
    q=[]
    for i in range(len(estado)):
        for j in range(len(estado)):
            if estado[i][j] == 1: q.append((i,j))
    return q

In [ ]:
# Teste
print(locateQueens(geraTabuleiro(4)))

In [26]:
def calAtaques(estado):
    at=0
    atlinhas=0
    # ataque em linhas
    for i in estado:
        if sum(i)>1:
            atlinhas+=sum(i)-1
    at+=atlinhas

    # ataques em colunas
    atcolunas = 0
    for c in range(len(estado)):
        n1 = 0
        for l in range(len(estado)): n1 += estado[l][c]
        if n1>1:
            atcolunas += n1 - 1
    at+=atcolunas
    pr = locateQueens(estado)
    d=0
    # Ataques em diagonais
    for p, coords in enumerate(pr):
        i,j = coords
        # diagonal esquerda
        k,w = i+1,j-1
        while(w>=0 and k<len(estado)):
            if estado[k][w]==1:
                d+=1
                break
            w-=1
            k+=1
        # diagonal direita
        l,c = i+1, j+1
        while(1 < len(estado) and c<len(estado)):
            if estado[1][c]==1:
                d+=1
                break
            l+=1
            c+=1
    at+=d
    return at


In [ ]:
# Teste da calculadora de ataques
print(calAtaques(geraTabuleiroAleatorio(4)))

In [28]:
def moveRainhaAleatoria(estado, passo):
    tam=len(estado)
    while True:
        i,j = random.choice(locateQueens(estado))
        k = (j+passo)%tam
        if estado[i][k] == 0:
            estado[i][k],estado[i][j] = estado[i][j],estado[i][k]
            break
    return estado

In [ ]:
# teste Move Rainha
printTabuleiro(moveRainhaAleatoria(geraTabuleiro(4),2))

In [30]:
def evolucaoFuncaoObjetivo(estado, passo, MAX):
#def evolucaoFuncaoObjetivo(estado, MAX):
    n = len(estado[0])
    ataques = []
    i = 0
    maxAtaques = math.factorial(n)/(2*math.factorial(n-2))
    while i < MAX:
        atq = calAtaques(estado)
        ataques.append(maxAtaques - atq)
        if atq == 0:
          break
        #passo = random.randint(1,n-1)
        estado = moveRainhaAleatoria(copy.deepcopy(estado), passo)
        i+=1
    x=range(len(ataques))
    plt.plot(x, ataques)
    plt.xlabel("Vizinhos Gerados")
    plt.ylabel("Função Objetivo")
    plt.title("Variação da Função Objetivo")
    #plt.legend(loc="lower right")
    plt.show()

In [ ]:
evolucaoFuncaoObjetivo(geraTabuleiro(8),1,1500)

In [32]:
# Busca Hill Climbing
def buscaHC(estado, passo, Max):
  c = 0
  n = len(estado[0])
  maxAtaques = math.factorial(n)/(2*math.factorial(n-2))
  ataques = []
  change = True
  while c <= Max:
    change=False
    atq = calAtaques(estado)
    ataques.append(maxAtaques - atq)
    if atq == 0: # é uma solução
      return "Solução", estado, atq, ataques
    suc  = moveRainhaAleatoria(copy.deepcopy(estado), passo)
    if calAtaques(suc) <= atq:
      estado = suc
      change = True
    if not change: c += 1
  return "Falha", estado, atq, ataques

In [ ]:
# Testando a busca hill climbing
m,e,a,y = buscaHC(geraTabuleiro(6),1,1500)
print("Função executada com",m)
print("ultimo estado analizado")
printTabuleiro(e)
x = range(len(y))
plt.plot(x,y)
plt.xlabel("Vizinhos Gerados")
plt.ylabel("Função Objetivo")
plt.show()

In [ ]:
# Hill Climbing estacastico (Reinicio Aleatório)
for i in range(100):
  t = geraTabuleiroAleatorio(8)
  m,e,a,y = buscaHC(t,1,500)
  if m == "Solução":
    print("Solução Encontrada no reinicio",i)
    printTabuleiro(e)
  else:
    print("{}".format(a), end=",")

In [35]:
# Tempera simulada
def temperaSimulada(estado, passo, temp):
  for tp in range(temp, 1, -1):
    at = calAtaques(estado)
    if at == 0:
      return "Solução", estado, at
    suc = moveRainhaAleatoria(copy.deepcopy(estado), passo)
    deltaE = at = calAtaques(suc)
    if deltaE >= 0:
      estado = suc
    else:
      n = random.randint(0,100)
      if n < math.exp(deltaE/tp): estado = suc
  return "Falha", estado, at

In [ ]:
m = "Falha"
c = 0
while m == "Falha":
  t = geraTabuleiroAleatorio(8)
  m, e, a = temperaSimulada(t, 1, 1000)
  c += 1

print("Rodadas:", c)
print("Função executada com",m)
print("Ataques:", a)
print("ultimo estado analizado")
printTabuleiro(e)

# Algoritmo Genético

In [37]:
def mutacao(estado, passo):
    tam=len(estado)
    while True:
        i,j = random.choice(locateQueens(estado))
        k = (j+passo)%tam
        if estado[i][k] == 0:
            estado[i][k],estado[i][j] = estado[i][j],estado[i][k]
            break
    return estado

In [38]:
# Gerar população inicial (N indivíduos de tamanho tam)
def geraPopulacaoInicial(N, tam):
    populacao = []
    for i in range(N):
        populacao.append(geraTabuleiroAleatorio(tam))
    return populacao

In [41]:
# Função de cruzamento
def crossover(genoma1, genoma2):
  g1 = copy.deepcopy(genoma1)
  g2 = copy.deepcopy(genoma2)
  corte = random.randint(0,len(g1)-1)
  return g1[:corte] + g2[corte:], g2[:corte] + g1[corte:] # dois filhos gerados

In [43]:
def fitness(populacao):
  fits = []
  tam = len(populacao[0])
  maxAtaques = math.factorial(tam)/(2*math.factorial(tam-2))
  for i in populacao:
    fits.append(1- calAtaques(i)/maxAtaques)
  return fits

In [44]:
# Testar Função fitness
pop = geraPopulacaoInicial(10,8)
print(fitness(pop))

[0.7857142857142857, 0.75, 0.5357142857142857, 0.6785714285714286, 0.6428571428571428, 0.8214285714285714, 0.75, 0.9285714285714286, 0.7857142857142857, 0.6785714285714286]


In [45]:
# Função da Roleta
def roletaViciada(populacao, fitness):
  # Levantamento da qualidade dos indivíduos
  totalFitness = sum(fitness)

  # calculando a fração de cada indivíduo na roleta
  fracoes = [f/totalFitness for f in fitness]

  # Criando os intervalos da roleta
  intervalos = []
  acum = 0
  for i in fracoes:
    acum += 1
    intervalos.append(round(acum, 2))

  # Girando a roleta
  resultadoRoleta = random.randint(0, len(populacao))/len(populacao)

  # Selecionando um individuo
  ind = 0
  while resultadoRoleta > intervalos[ind]:
    ind += 1

  # Retorna o indivíduo selecionado
  return populacao[ind]

In [ ]:
pop = geraPopulacaoInicial(10,8)
fit = fitness(pop)
printTabuleiro(roletaViciada(pop, fit))

In [48]:
# Função para seleção natural, usando a roleta para escolher os reprodutores
def selecaoNatural(populacao, numCasais):
  selecao = []
  fts = fitness(populacao)
  for i in range(2*numCasais):
    selecao.append(roletaViciada(populacao, fts))
  return selecao

In [49]:
def testeMeta(pops):
  fts = fitness(pops)
  try:
    posMeta = fts.index(1)
    return "Solução", pops[posMeta]
  except (ValueError):
    return -1

In [50]:
# Função de algoritmo genético de fato
def genetico(maxIteracoes, tamanhoPopulacao, tamGenoma, taxaReproducao, deslocamentoGene):

  # Gerar população inicial
  populacao = geraPopulacaoInicial(tamanhoPopulacao, tamGenoma)

  # Iterações
  for i in range(maxIteracoes):
    # Primeiro teste de meta
    res = testeMeta(populacao)
    if res != -1: return res, i
    # Etapa de evolucao por rerodução
    novaPopulacao = []
    reprodutores = selecaoNatural(populacao, taxaReproducao)
    machos = reprodutores[:taxaReproducao]
    femeas = reprodutores[taxaReproducao:]
    for k, w in zip(machos, femeas):
      f1, f2 = crossover(k, w)
      novaPopulacao.append(f1)
      novaPopulacao.append(f2)
    # Teste de meta na nova população
    res = testeMeta(novaPopulacao)
    if res != -1: return res, i

    # Aceitar mutações na população
    for z in novaPopulacao:
      if random.randint(0,100) < 10:
        z=mutacao(z, deslocamentoGene)
    populacao = novaPopulacao
  return "Falha", populacao

In [52]:
# Teste de algoritmo genético
maxIteracoes = 1000
tamanhoPopulacaoInicial = 100
tamGenoma = 4 # tabuleiro n x n
casaisReprodutores = 40 # numero de casais a serem formados para reprodução
deslocamentoGene = 1 # passo no tabuleiro -> regra de mutacao do genoma

genetico(maxIteracoes, tamanhoPopulacaoInicial, tamGenoma, casaisReprodutores, deslocamentoGene)

(('Solução', [[0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0]]), 822)